In [620]:
import pandas as pd
from pathlib import Path  
import sys
import sqlite3

CUT_OF_UNIQUE_VALUES=40

In [621]:
data_filtered_cedd=pd.read_csv("cleaned_data_with_binary_dead.csv")
df_counts  = pd.DataFrame(columns = ['col', 'count'])
for col in data_filtered_cedd:
    no_of_unique_values=data_filtered_cedd[col].nunique()
    df_counts.loc[len(df_counts)]=[col,no_of_unique_values]
#     if( not col.startswith( 'Unnamed' ) && no_of_unique_values<20):
#         print("%s--->%s"%(col,))
df_counts = df_counts.sort_values(by=['count'], ascending=False)
display(df_counts.head(30))

count=0

for col in data_filtered_cedd:
    no_of_unique_values=data_filtered_cedd[col].nunique()
    if( not col.startswith( 'Unnamed' ) and no_of_unique_values<40):
        count =count+1
        #data_filtered_cedd[col].fillna("imputed",inplace=True)
        data_filtered_cedd[col]=pd.Categorical(data_filtered_cedd[col])
                
data_filtered_cedd.dtypes.value_counts()



count


    



C:\Users\sjyhl\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (75,83,88,136) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,col,count
0,Unnamed: 0,28122
2,Unnamed: 0.1.1,28122
1,Unnamed: 0.1,28122
29,ceo_id,15744
28,epi_id,7432
4,pa_id,5930
13,epi_SocDate,1056
22,icd_code,967
23,icd_code_description,965
14,epi_StartOfEpisode,833


133

In [622]:
data_filtered_cedd["M1056_PVX_RSN_NOT_RCVD_AGNCY"].cat.categories
data_filtered_cedd.groupby(['M1056_PVX_RSN_NOT_RCVD_AGNCY']).size()

data_filtered_cedd["M1200_VISION"].cat.categories
data_filtered_cedd.groupby(['M1200_VISION']).size()

M1200_VISION
0 - NORMAL VISION: SEES ADEQUATELY IN MOST SITUATIONS; CAN SEE MEDICATION LABELS, NEWSPRINT                                                                        11108
1 - PARTIALLY IMPAIRED: CANNOT SEE MEDICATION LABELS OR NEWSPRINT, BUT CAN SEE OBSTACLES IN PATH, AND THE SURROUNDING LAYOUT; CAN COUNT FINGERS AT ARM'S LENGTH     3545
2 - SEVERELY IMPAIRED: CANNOT LOCATE OBJECTS WITHOUT HEARING OR TOUCHING THEM OR PATIENT NONRESPONSIVE                                                               197
dtype: int64

### Drop columns with no meaning

In [623]:
data_filtered_cedd = data_filtered_cedd.drop(columns=['CreatedDate', 'CreatedBy', 'UpdateDate', 'UpdateBy', 'ActiveFlag'])

In [624]:
data_filtered_cedd['DaysToDeath']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
28117    2.0
28118    2.0
28119    2.0
28120    2.0
28121    2.0
Name: DaysToDeath, Length: 28122, dtype: float64

### Find columns with name ID

In [625]:
[col for col in data_filtered_cedd.columns if 'id' in col]

['pa_id', 'epi_dcrid', 'epi_id', 'ceo_id']

In [626]:
[col for col in data_filtered_cedd.columns if 'Id' in col]

['DispositionId', 'amIdead?']

### Drop ID columns

In [627]:
df = data_filtered_cedd.drop(columns=['DispositionId', 'pa_id', 'epi_dcrid', 'epi_id', 'ceo_id'])

### Drop records with DaysToDeath = 0

In [628]:
df.dropna(subset=['DaysToDeath'])

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,year_born,pa_race,pa_gender,pa_maritalstatus,epi_AdmitType,dcc_desc,dcr_desc,...,M2310_ECR_UNKNOWN,M2401_INTRVTN_SMRY_DBTS_FT,M2401_INTRVTN_SMRY_DPRSN,M2401_INTRVTN_SMRY_FALL_PRVNT,M2401_INTRVTN_SMRY_PAIN_MNTR,M2401_INTRVTN_SMRY_PRSULC_PRVN,M2401_INTRVTN_SMRY_PRSULC_WET,M2410_INPAT_FACILITY,M2420_DSCHRG_DISP,amIdead?
40,40,40,40,1930,NaN,MALE,NaN,RECERTIFICATION,PATIENT EXPIRED,PATIENT EXPIRED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
41,41,41,41,1930,NaN,MALE,NaN,RECERTIFICATION,PATIENT EXPIRED,PATIENT EXPIRED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
42,42,42,42,1930,NaN,MALE,NaN,RECERTIFICATION,PATIENT EXPIRED,PATIENT EXPIRED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
43,43,43,43,1930,NaN,MALE,NaN,RECERTIFICATION,PATIENT EXPIRED,PATIENT EXPIRED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
76,76,76,76,1966,NaN,FEMALE,NaN,NEW ADMISSION,PATIENT EXPIRED,PATIENT EXPIRED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28117,28117,28117,28117,1940,NaN,FEMALE,NaN,NEW ADMISSION,PATIENT EXPIRED,PATIENT EXPIRED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
28118,28118,28118,28118,1940,NaN,FEMALE,NaN,NEW ADMISSION,PATIENT EXPIRED,PATIENT EXPIRED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
28119,28119,28119,28119,1940,NaN,FEMALE,NaN,NEW ADMISSION,PATIENT EXPIRED,PATIENT EXPIRED,...,NO,NA - PATIENT IS NOT DIABETIC OR IS MISSING LOW...,NA - (C2) PATIENT HAS NO DIAGNOSIS OF DEPRESSI...,1 - YES,1 - YES,1 - YES,NA - PATIENT HAS NO PRESSURE ULCERS OR HAS NO ...,1 - HOSPITAL,NaN,1
28120,28120,28120,28120,1940,NaN,FEMALE,NaN,NEW ADMISSION,PATIENT EXPIRED,PATIENT EXPIRED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [629]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df.select_dtypes(include=numerics)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,year_born,epi_height,epi_weight,DaysToDeath,RoughAge
0,0,0,0,1940,NaN,NaN,NaN,NaN
1,1,1,1,1940,NaN,NaN,NaN,NaN
2,2,2,2,1940,NaN,NaN,NaN,NaN
3,3,3,3,1940,NaN,NaN,NaN,NaN
4,4,4,4,1961,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
28117,28117,28117,28117,1940,NaN,NaN,2.0,82.0
28118,28118,28118,28118,1940,NaN,NaN,2.0,82.0
28119,28119,28119,28119,1940,NaN,NaN,2.0,82.0
28120,28120,28120,28120,1940,NaN,NaN,2.0,82.0


In [630]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

### Also drop dcc_desc as dcr_desc should be more accurate representation of discharge info

In [697]:
df = df.drop(columns=['dcc_desc'])

In [698]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import numpy as np

imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='imputed')
idf = pd.DataFrame(imputer.fit_transform(df))
idf.columns=df.columns
# idf.index=df.index

#
df = idf
for col in df:
    no_of_unique_values=df[col].nunique()
    if( not col.startswith( 'Unnamed' ) and no_of_unique_values<40):        
        df[col]=pd.Categorical(df[col])

In [699]:
df.dtypes

year_born                           object
pa_race                           category
pa_gender                         category
pa_maritalstatus                  category
epi_AdmitType                     category
                                    ...   
M2401_INTRVTN_SMRY_PRSULC_PRVN    category
M2401_INTRVTN_SMRY_PRSULC_WET     category
M2410_INPAT_FACILITY              category
M2420_DSCHRG_DISP                 category
amIdead?                          category
Length: 136, dtype: object

#### detect columns that contain numeric value when it shouldn't be. (This runs quite a long time)

In [700]:
for column_name in df.columns:
    if column_name != 'year_born':
        for index, row in df.iterrows():
            if isinstance(row[column_name], (int, float)):
                print(column_name)
                df[column_name] = df[column_name].astype(str)
                break

In [636]:
# df = df.drop(columns=['epi_SocDate', 'epi_StartOfEpisode', 'epi_EndOfEpisode', 'icd_code_description', 'epi_DischargeDate', 'M0090_INFO_COMPLETED_DT'])

In [702]:
# Use Ordinal Encoder for all numeric type category

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(df)
enc.categories_

[array([1909, 1913, 1914, 1915, 1917, 1918, 1919, 1920, 1921, 1922, 1923,
        1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934,
        1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945,
        1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956,
        1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967,
        1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
        1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
        1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2001,
        2011, 2019], dtype=object),
 array(['1.0', '2.0', '3.0', '5.0', '6.0', '7.0', '8.0', 'imputed'],
       dtype=object),
 array(['FEMALE', 'MALE  '], dtype=object),
 array(['1.0', '2.0', '25000.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0',
        'imputed'], dtype=object),
 array(['NEW ADMISSION', 'READMISSION', 'RECERTIFICATION'], dtype=object),
 array(['CLIENT NO LONGER REQUIRES SKILLED CARE', 'D

In [703]:
xdf = enc.transform(df) # transform using encoder

In [704]:
df2 = pd.DataFrame(xdf, columns=df.columns) # numpy to dataframe

In [705]:
X = df2.iloc[:,:-1]

In [706]:
y = df2.iloc[:,-1:]

In [707]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [727]:
X_train

,year_born,pa_race,pa_gender,pa_maritalstatus,epi_AdmitType,dcr_desc,epi_height,epi_weight,ceo_HIPPS,ceo_HHRG,...,M2310_ECR_OTHER,M2310_ECR_UNKNOWN,M2401_INTRVTN_SMRY_DBTS_FT,M2401_INTRVTN_SMRY_DPRSN,M2401_INTRVTN_SMRY_FALL_PRVNT,M2401_INTRVTN_SMRY_PAIN_MNTR,M2401_INTRVTN_SMRY_PRSULC_PRVN,M2401_INTRVTN_SMRY_PRSULC_WET,M2410_INPAT_FACILITY,M2420_DSCHRG_DISP
4534,31.0,7.0,0.0,9.0,0.0,13.0,72.0,181.0,226.0,39.0,...,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,5.0
24447,23.0,7.0,0.0,9.0,0.0,14.0,72.0,181.0,116.0,13.0,...,1.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,0.0,5.0
9918,46.0,7.0,0.0,1.0,0.0,17.0,72.0,181.0,93.0,36.0,...,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0
21862,28.0,7.0,0.0,9.0,0.0,14.0,72.0,181.0,226.0,39.0,...,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0
24836,36.0,7.0,0.0,9.0,0.0,14.0,72.0,181.0,226.0,39.0,...,1.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,61.0,7.0,0.0,9.0,2.0,14.0,72.0,181.0,226.0,39.0,...,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0
5390,14.0,7.0,0.0,9.0,0.0,8.0,72.0,181.0,226.0,39.0,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,4.0,0.0
860,44.0,7.0,0.0,9.0,0.0,8.0,72.0,181.0,80.0,34.0,...,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,0.0
15795,30.0,7.0,0.0,9.0,0.0,0.0,72.0,181.0,226.0,39.0,...,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0


### Feature Selection: LASSO

In [708]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

lasso_model = LassoCV(cv=5, normalize=True)

In [709]:
sfm = SelectFromModel(lasso_model, threshold=0.25, prefit=False, norm_order=1, max_features=None)
sfm.fit(X_train, y_train)
feature_idx = sfm.get_support()
df.columns[:-1][feature_idx]

C:\Users\sjyhl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sjyhl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original

Index([], dtype='object')

In [710]:
from numpy import mean
from numpy import std
from numpy import absolute

scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE: 0.199 (0.003)


In [718]:
#df.columns[:-1]
sfm.transform(X_train).shape[1]

C:\Users\sjyhl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_base.py:92: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warn(


0

In [714]:
vars(sfm)

{'estimator': LassoCV(cv=5, normalize=True),
 'threshold': 0.25,
 'prefit': False,
 'importance_getter': 'auto',
 'norm_order': 1,
 'max_features': None,
 'estimator_': LassoCV(cv=5, normalize=True),
 'feature_names_in_': array(['year_born', 'pa_race', 'pa_gender', 'pa_maritalstatus',
        'epi_AdmitType', 'dcr_desc', 'epi_height', 'epi_weight',
        'ceo_HIPPS', 'ceo_HHRG', 'ceas_ascode', 'icd_code',
        'DiagnosisCategory', 'cedd_dateofdeath', 'DaysToDeath', 'RoughAge',
        'ceo_OASIScorrectionnum', 'M0100_ASSMT_REASON',
        'M0110_EPISODE_TIMING', 'M0140_ETHNIC_AI_AN', 'M0140_ETHNIC_ASIAN',
        'M0140_ETHNIC_BLACK', 'M0140_ETHNIC_HISP', 'M0140_ETHNIC_NH_PI',
        'M0140_ETHNIC_WHITE', 'M0906_DC_TRAN_DTH_DT',
        'M1000_DC_IPPS_14_DA', 'M1000_DC_IRF_14_DA', 'M1000_DC_LTC_14_DA',
        'M1000_DC_LTCH_14_DA', 'M1000_DC_NONE_14_DA', 'M1000_DC_OTH_14_DA',
        'M1000_DC_PSYCH_14_DA', 'M1000_DC_SNF_14_DA', 'M1028_ACTV_DIAG_DM',
        'M1028_ACTV_DIAG_NO

In [ ]:
df.columns[:-1][np.abs(lasso_model.coef_) > 0]

### Misc Facts about data

#### some columns have '-' as category

In [614]:
df.M1028_ACTV_DIAG_DM.unique() 

['imputed', 'YES', 'NO', '-']
Categories (4, object): ['-', 'NO', 'YES', 'imputed']

In [159]:
df.M1028_ACTV_DIAG_PVD_PAD.unique() 

['imputed', 'YES', 'NO', '-']
Categories (4, object): ['-', 'NO', 'YES', 'imputed']

In [202]:
df.M2003_MDCTN_FLWP.unique() 

['imputed', '0 - NO', '1 - YES', '-']
Categories (4, object): ['-', '0 - NO', '1 - YES', 'imputed']

#### some columns represent single variable type, some are over multiple columns

In [109]:
list(df.M1400_WHEN_DYSPNEIC.unique())

['2 - WITH MODERATE EXERTION (FOR EXAMPLE, WHILE DRESSING, USING COMMODE OR BEDPAN, WALKING DISTANCES LESS THAN 20 FEET)',
 '1 - WHEN WALKING MORE THAN 20 FEET, CLIMBING STAIRS ',
 '0 - PATIENT IS NOT SHORT OF BREATH ',
 '4 - AT REST (DURING DAY OR NIGHT)',
 '3 - WITH MINIMAL EXERTION (FOR EXAMPLE, WHILE EATING, TALKING, OR PERFORMING OTHER ADLS) OR WITH AGITATION',
 'imputed']

In [110]:
list(df.M1033_HOSP_RISK_MLTPL_HOSPZTN.unique())

['imputed', 'NO', 'YES']

### Feature Evaluation using Regression

In [719]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

names=pd.DataFrame(X_train.columns)

lin_reg = LinearRegression()

rfe_mod = RFE(lin_reg, n_features_to_select=5, step=1)

fit_model = rfe_mod.fit(X_train, y_train)
fit_model.support_
fit_model.ranking_

array([ 88,  64,  28,  72,  12,  27, 101, 122, 130, 123,  91, 131,  74,
       109,  99,  59,  20, 120,  86,   1,   2,  16,  37,   3,  84, 119,
        87,  79,  45, 124,   5,  58,  56,   4,  62,  94,  61,  82,  57,
       116,   6,  23,  76, 113,  17, 125, 108,  15,   1,  98,   1,  93,
        29,  30, 128, 129,  44,  46,   1,  52, 104,  81,  69,  70, 110,
        90,  80,  42,  19,  95,  51,  41, 100,  63,  49, 117, 111, 112,
        13, 114,  35,  18,  68,  89,  54,  60,  33,  36,  24,  25,  11,
        97,  66,  39,  67,  48,  55,  96, 121,   8,  47, 102, 105,  31,
        32,  83, 103,  75, 118,  10,  92, 115, 107,   9,  85,  53,  71,
        40,  34,  78,   7, 126,  65,  43,  21,  26,  22,  38,  50,   1,
        77, 106,  73, 127,  14])

In [721]:
selected_features = []
for i, rank in enumerate(fit_model.ranking_):
    feature = fit_model.feature_names_in_[i]
    selected_features.append({'feature': feature, 'rank': rank})
selected_features

[{'feature': 'year_born', 'rank': 88},
 {'feature': 'pa_race', 'rank': 64},
 {'feature': 'pa_gender', 'rank': 28},
 {'feature': 'pa_maritalstatus', 'rank': 72},
 {'feature': 'epi_AdmitType', 'rank': 12},
 {'feature': 'dcr_desc', 'rank': 27},
 {'feature': 'epi_height', 'rank': 101},
 {'feature': 'epi_weight', 'rank': 122},
 {'feature': 'ceo_HIPPS', 'rank': 130},
 {'feature': 'ceo_HHRG', 'rank': 123},
 {'feature': 'ceas_ascode', 'rank': 91},
 {'feature': 'icd_code', 'rank': 131},
 {'feature': 'DiagnosisCategory', 'rank': 74},
 {'feature': 'cedd_dateofdeath', 'rank': 109},
 {'feature': 'DaysToDeath', 'rank': 99},
 {'feature': 'RoughAge', 'rank': 59},
 {'feature': 'ceo_OASIScorrectionnum', 'rank': 20},
 {'feature': 'M0100_ASSMT_REASON', 'rank': 120},
 {'feature': 'M0110_EPISODE_TIMING', 'rank': 86},
 {'feature': 'M0140_ETHNIC_AI_AN', 'rank': 1},
 {'feature': 'M0140_ETHNIC_ASIAN', 'rank': 2},
 {'feature': 'M0140_ETHNIC_BLACK', 'rank': 16},
 {'feature': 'M0140_ETHNIC_HISP', 'rank': 37},
 {'f

In [722]:
sorted(selected_features, key=lambda i: i['rank'])

[{'feature': 'M0140_ETHNIC_AI_AN', 'rank': 1},
 {'feature': 'M1033_HOSP_RISK_NONE_ABOVE', 'rank': 1},
 {'feature': 'M1033_HOSP_RISK_WEIGHT_LOSS', 'rank': 1},
 {'feature': 'M1306_UNHLD_STG2_PRSR_ULCR', 'rank': 1},
 {'feature': 'M2401_INTRVTN_SMRY_FALL_PRVNT', 'rank': 1},
 {'feature': 'M0140_ETHNIC_ASIAN', 'rank': 2},
 {'feature': 'M0140_ETHNIC_NH_PI', 'rank': 3},
 {'feature': 'M1000_DC_SNF_14_DA', 'rank': 4},
 {'feature': 'M1000_DC_NONE_14_DA', 'rank': 5},
 {'feature': 'M1030_THH_PAR_NUTRITION', 'rank': 6},
 {'feature': 'M2200_THER_NEED_NA', 'rank': 7},
 {'feature': 'M1800_CRNT_GROOMING', 'rank': 8},
 {'feature': 'M2016_DRUG_EDCTN_INTRVTN', 'rank': 9},
 {'feature': 'M2001_DRUG_RGMN_RVW', 'rank': 10},
 {'feature': 'M1730_STDZ_DPRSN_SCRNG', 'rank': 11},
 {'feature': 'epi_AdmitType', 'rank': 12},
 {'feature': 'M1340_SRGCL_WND_PRSNT', 'rank': 13},
 {'feature': 'M2420_DSCHRG_DISP', 'rank': 14},
 {'feature': 'M1033_HOSP_RISK_MNTL_BHV_DCLN', 'rank': 15},
 {'feature': 'M0140_ETHNIC_BLACK', 'ran